In [1]:
import copy
import torch
import numpy as np
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import importlib
import os
import time
from scipy.spatial.transform import Rotation
from scipy.spatial.distance import cdist
import poisson_modelnet_40
importlib.reload(poisson_modelnet_40)

from poisson_modelnet_40 import (
    get_transform,
    get_rotation_transform,
    get_single_modelnet40_sample
)

import pose_estimation
importlib.reload(pose_estimation)
from pose_estimation import ICP, PointCloudMetropolisHastings, nearest_neighbor_src_dst, metrics_per_step

import tbp.monty.frameworks.environment_utils.transforms
importlib.reload(tbp.monty.frameworks.environment_utils.transforms)
import tbp.monty.frameworks.environments.modelnet
importlib.reload(tbp.monty.frameworks.environments.modelnet)

from tbp.monty.frameworks.environment_utils.transforms import RandomRotate
from tbp.monty.frameworks.utils.metrics import TransformedPointCloudDistance
from tbp.monty.frameworks.environments.modelnet import ModelNet40OnlineOptimization



# Step 1: just see if we can implement ICP or similar.

- Load a single modelnet40 object.
- Sample k points from the mesh.
- Load the same object and apply a known rotation.
- Call pose_estimator(src, tgt)
- Decode the output of pose_estimator so we can compare to known transform
- Examine the fit of the learned transform
- Measure how much time passed
- Extend by replacing error_fn with poisson surface reconstruction

In [45]:
N_SAMPLES = 1024
dst_transform = get_transform(N_SAMPLES)
rot_transform = RandomRotate(axes=["y"], fix_rotation=True)
rotation_matrix = rot_transform.rotation_matrix
src_transform = T.Compose([dst_transform, rot_transform])
dataset = ModelNet40OnlineOptimization(
    root=os.path.expanduser("~/tbp/datasets/ModelNet40/raw"),
    transform=None,  # raw torch geometric object
    train=True,
    num_samples_train=2,
    dst_transform=dst_transform,
    src_transform=rot_transform
)
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=1)

In [46]:
x0_dst = dst_transform(dataset.data[0])
print(x0_dst.size())  # N x 3; need to take transpose to so we can multiply correctly

torch.Size([1024, 3])


In [47]:
e0 = np.random.uniform(0, 2 * np.pi)
r0 = Rotation.from_euler("x", e0).as_matrix()
r0_torch = torch.from_numpy(r0).float()
r0_torch_inv = torch.inverse(r0_torch)
r0_torch_t_inv = torch.inverse(r0_torch.T)

Show that when X is N x 3, RX.T is not the same as XR. RX.T is the correct way according to wikipedia. If you want to post-multiply instead of pre-multiply, then using the rules of algebra: (RX.T).T = XR.T, since you reverse the order and apply the transpose to both.

In [48]:
x0_src_L = torch.matmul(r0_torch, x0_dst.T)  # proposed way of doing it, correct according to wikipedia
x0_src_R = torch.matmul(x0_dst, r0_torch)  # original way of doing it
x0_src_R_t = torch.matmul(x0_dst, r0_torch.T)  # corrected version of original
n_entries = x0_dst.size(0) * x0_dst.size(1)

print(torch.isclose(x0_src_L, x0_src_R.T).sum() == n_entries)
print(torch.isclose(x0_src_L, x0_src_R_t.T).sum() == n_entries)



tensor(False)
tensor(True)


In [49]:
x0_est_L_inv = torch.matmul(r0_torch_inv, x0_src_L)  # show we can invert the transform
assert torch.isclose(x0_est_L_inv, x0_dst.T).sum() == n_entries